# Iterative Prompt Development

In [2]:
import os
from openai import OpenAI

In [55]:
api_key='YOUR API KEY'

In [4]:
#call the API
from openai import OpenAI

client = OpenAI(api_key=api_key)

use 'gpt-3.5-turbo' model and chat completions endpoint for the following examples.

In [5]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": prompt,
            }
        ],
        model=model,
        temperature=0

    )
    return chat_completion.choices[0].message.content

## Generate a marketing product description from a product fact sheet

We have a fact sheet for chairs and we want to extract some informations from this sheet. By iterating the prompt we will have the targeted result. 

In [8]:
fact_sheet_chair = """
OVERVIEW
- Part of a beautiful family of mid-century inspired office furniture, 
including filing cabinets, desks, bookcases, meeting tables, and more.
- Several options of shell color and base finishes.
- Available with plastic back and front upholstery (SWC-100) 
or full upholstery (SWC-110) in 10 fabric and 6 leather options.
- Base finish options are: stainless steel, matte black, 
gloss white, or chrome.
- Chair is available with or without armrests.
- Suitable for home or business settings.
- Qualified for contract use.

CONSTRUCTION
- 5-wheel plastic coated aluminum base.
- Pneumatic chair adjust for easy raise/lower action.

DIMENSIONS
- WIDTH 53 CM | 20.87”
- DEPTH 51 CM | 20.08”
- HEIGHT 80 CM | 31.50”
- SEAT HEIGHT 44 CM | 17.32”
- SEAT DEPTH 41 CM | 16.14”

OPTIONS
- Soft or hard-floor caster options.
- Two choices of seat foam densities: 
 medium (1.8 lb/ft3) or high (2.8 lb/ft3)
- Armless or 8 position PU armrests 

MATERIALS
SHELL BASE GLIDER
- Cast Aluminum with modified nylon PA6/PA66 coating.
- Shell thickness: 10 mm.
SEAT
- HD36 foam

COUNTRY OF ORIGIN
- Italy
"""

In [9]:
prompt = f"""
Your task is to help a marketing team create a 
description for a retail website of a product based 
on a technical fact sheet.

Write a product description based on the information 
provided in the technical specifications delimited by 
triple backticks.

Technical specifications: ```{fact_sheet_chair}```
"""
response = get_completion(prompt)
print(response)


Introducing the SWC-100/SWC-110 Office Chair, a sleek and stylish addition to any home or business setting. This chair is part of a stunning collection of mid-century inspired office furniture, offering a cohesive look with filing cabinets, desks, bookcases, meeting tables, and more.

Customize your SWC-100/SWC-110 Office Chair with several options of shell color and base finishes to suit your personal style. Choose between plastic back and front upholstery or full upholstery in a variety of fabric and leather options. The base finish options include stainless steel, matte black, gloss white, or chrome, allowing you to create the perfect look for your space.

Constructed with a 5-wheel plastic coated aluminum base and featuring a pneumatic chair adjust for easy raise/lower action, this chair combines both style and functionality. The dimensions of the chair are as follows: WIDTH 53 CM, DEPTH 51 CM, HEIGHT 80 CM, SEAT HEIGHT 44 CM, and SEAT DEPTH 41 CM.

Additional options for the SWC-1

it took few seconds to run and it looks like done a good job. 
### ISSUE 1: LONG TEXT
The response too long. to make the result less longer we should clarify a new rule in in the prompt: 'Use at most 50 words. 
'

In [10]:
prompt = f"""
Your task is to help a marketing team create a 
description for a retail website of a product based 
on a technical fact sheet.

Write a product description based on the information 
provided in the technical specifications delimited by 
triple backticks.

Use at most 50 words. 

Technical specifications: ```{fact_sheet_chair}```
"""
response = get_completion(prompt)
print(response)


Introducing our versatile and stylish mid-century office chair, available in a variety of colors and finishes. With adjustable height and comfortable upholstery options, this chair is perfect for both home and business use. Made in Italy with high-quality materials for durability. Elevate your workspace today!


In [11]:
#Let's check the lenght of the response

In [12]:
len(response.split(' '))

45

We can limit the lenght by number of sentences.

In [13]:
prompt = f"""
Your task is to help a marketing team create a 
description for a retail website of a product based 
on a technical fact sheet.

Write a product description based on the information 
provided in the technical specifications delimited by 
triple backticks.

Use only 5 sentences. 

Technical specifications: ```{fact_sheet_chair}```
"""
response = get_completion(prompt)
print(response)


Introducing our versatile and stylish mid-century inspired office chair, perfect for both home and business settings. With a variety of shell colors, base finishes, and upholstery options, you can customize this chair to fit your unique style. The 5-wheel plastic coated aluminum base and pneumatic chair adjust make for easy mobility and height adjustment. Choose between soft or hard-floor casters, two seat foam densities, and armless or PU armrest options for maximum comfort. Crafted with quality materials like cast aluminum and HD36 foam, this chair is not only aesthetically pleasing but also durable. Made in Italy, this chair is designed to elevate any workspace with its modern design and functionality.


The limitation can be done by sentences, words or characters.

## ISSUE 2: WRONG DETAILS
- Ask it to focus on the aspects that are relevant to the intended audience.

In [16]:
prompt = f"""
Your task is to help a marketing team create a 
description for a retail website of a product based 
on a technical fact sheet.

Write a product description based on the information 
provided in the technical specifications delimited by 
triple backticks.

The description is intended for furniture retailers, 
so should be technical in nature and focus on the 
materials the product is constructed from.

Use at most 50 words.

Technical specifications: ```{fact_sheet_chair}```
"""
response = get_completion(prompt)
print(response)

Introducing our versatile and stylish office chair, part of a mid-century inspired furniture collection. Constructed with a durable cast aluminum shell and base glider coated with modified nylon PA6/PA66, this chair offers comfort with HD36 foam seat. Available in various colors and finishes, suitable for both home and business use. Made in Italy.


we want to add the product IDs too.

In [17]:
prompt = f"""
Your task is to help a marketing team create a 
description for a retail website of a product based 
on a technical fact sheet.

Write a product description based on the information 
provided in the technical specifications delimited by 
triple backticks.

The description is intended for furniture retailers, 
so should be technical in nature and focus on the 
materials the product is constructed from.

At the end of the description, include every 7-character 
Product ID in the technical specification.

Use at most 50 words.

Technical specifications: ```{fact_sheet_chair}```
"""
response = get_completion(prompt)
print(response)

Introducing our versatile office chair, part of a stylish mid-century inspired collection. Choose from a variety of shell colors and base finishes to suit your space. Constructed with a durable aluminum base and high-density foam seat for comfort. Perfect for home or office use. Product IDs: SWC-100, SWC-110.


## ISSUE 3. TABLE OF DIMENSIONS
- Ask it to extract information and organize it in a table.

In [19]:
prompt = f"""
Your task is to help a marketing team create a 
description for a retail website of a product based 
on a technical fact sheet.

Write a product description based on the information 
provided in the technical specifications delimited by 
triple backticks.

The description is intended for furniture retailers, 
so should be technical in nature and focus on the 
materials the product is constructed from.

At the end of the description, include every 7-character 
Product ID in the technical specification.

After the description, include a table that gives the 
product's dimensions. The table should have two columns.
In the first column include the name of the dimension. 
In the second column include the measurements in inches only.

Give the table the title 'Product Dimensions'.

Format everything as HTML that can be used in a website. 
Place the description in a <div> element.

Technical specifications: ```{fact_sheet_chair}```
"""

response = get_completion(prompt)
print(response)

<div>
<p>Introducing our latest addition to the mid-century inspired office furniture collection - the versatile and stylish office chair. This chair is not only aesthetically pleasing but also designed for comfort and durability.</p>

<p>The chair features a 5-wheel plastic coated aluminum base for stability and mobility. The pneumatic chair adjust allows for easy raise and lower action, ensuring optimal seating position for any user.</p>

<p>Choose from a variety of shell colors and base finishes to match your office decor. The chair is available with plastic back and front upholstery or full upholstery in a range of fabric and leather options. You can also customize the chair with or without armrests to suit your preference.</p>

<p>Constructed with high-quality materials, the shell base glider is made of cast aluminum with a modified nylon PA6/PA66 coating, providing strength and durability. The seat is filled with HD36 foam for maximum comfort during long hours of work.</p>

<p>Th

LET'S SEE THE RESULT

In [20]:
from IPython.display import display, HTML

In [21]:
display(HTML(response))

Dimension,Measurements (inches)
Width,"20.87"""
Depth,"20.08"""
Height,"31.50"""
Seat Height,"17.32"""
Seat Depth,"16.14"""
